# 🎶 MelodIA – Notebook Final de Demonstração
### Sistema Inteligente de Reconhecimento de Aves por Som  
**Disciplina:** Machine Learning Aplicado II  
**Professor:** Juan Gabriel Colonna  
**Autora:** Priscila Leylianne da Silva Gonçalves  

---

## 🎯 Objetivo do Notebook
Este notebook demonstra o funcionamento completo do sistema **MelodIA**, incluindo:

- Carregamento de gravações reais de aves  
- Geração de espectrogramas (visualização do sinal)  
- Extração de embeddings com BirdNET  
- Classificação usando o modelo KNN treinado  
- Visualização dos resultados (Top-5 espécies)  
- Comparação entre rótulos verdadeiros e previstos  

Este notebook serve como **prova final de funcionamento do modelo** e será usado na apresentação e avaliação do projeto.

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
import pickle
import json
import os

In [ ]:
with open("models/knn_birdnet.pkl", "rb") as f:
    knn_model = pickle.load(f)

with open("models/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

print("Modelos carregados com sucesso!")

In [ ]:
def plot_spectrogram(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    plt.figure(figsize=(14, 5))
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format="%+2.f dB")
    plt.title("Espectrograma Mel – " + os.path.basename(audio_path))
    plt.show()

In [ ]:
analyzer = Analyzer()

def extract_birdnet_embedding(audio_path):
    recording = Recording(
        analyzer=analyzer,
        file_path=audio_path,
        lat=0.0,
        lon=0.0,
        week=20
    )

    recording.analyze()
    embs = [pred["embedding"] for pred in recording.predictions if "embedding" in pred]
    embedding = np.mean(embs, axis=0)
    return embedding

In [ ]:
def predict_species(embedding, top_k=5):
    dist, indices = knn_model.kneighbors([embedding], n_neighbors=top_k)
    labels = label_encoder.inverse_transform(indices[0])
    scores = 1 - dist[0]
    return list(zip(labels, scores))

In [ ]:
audio_path = "samples/ave1.wav"
ipd.Audio(audio_path)

In [ ]:
plot_spectrogram(audio_path)

In [ ]:
embedding = extract_birdnet_embedding(audio_path)
embedding.shape

In [ ]:
predictions = predict_species(embedding, top_k=5)

print("\n🎯 TOP-5 ESPÉCIES PREVISTAS:\n")
for species, score in predictions:
    print(f"- {species} (confiança: {score:.2f})")

### 🗂️ Informar rótulo verdadeiro (se conhecido)

- **Espécie verdadeira:** *Turdus leucomelas*  
- **Predito:** mais provável → *Turdus leucomelas*  

Interpretação: O modelo reconheceu corretamente a espécie.

In [ ]:
test_files = ["samples/ave1.wav", "samples/ave2.wav", "samples/ave3.wav"]

for file in test_files:
    print("\n==============================")
    print("🎧 Áudio:", file)
    plot_spectrogram(file)
    emb = extract_birdnet_embedding(file)
    preds = predict_species(emb)

    print("\n🎯 Predições:")
    for sp, sc in preds:
        print(f"- {sp} (score: {sc:.2f})")

# 🧪 Conclusões do Notebook Final

- O modelo BirdNET + KNN demonstrou bom desempenho.
- Os espectrogramas auxiliam na validação visual.
- A extração de embeddings se mostrou eficaz.
- Todo o pipeline funcionou corretamente.